In [4]:
import json
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from pymilvus import connections, db
from dotenv import load_dotenv
import os
import time
import json
from tqdm import tqdm
load_dotenv()

True

## STEP-1 Prepare Documents

In [5]:
with open('../documents.json') as file:
    data = json.load(file)

In [141]:
course_name = data[0]['course']
i = 1
for document in data[0]['documents']:
    document['document_id'] = i
    document['course'] = course_name
    i = i+1

In [8]:
json_data = data[0]['documents']

In [9]:
json_data[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

In [145]:
len(json_data)

435

In [146]:
with open('data.json', 'w') as f_out:
    json.dump(json_data, f_out) 

### Creating Ground Truth dataset

In [16]:
OpenAI_KEY = os.getenv("OPEN_AI_API_KEY")

In [147]:
client = OpenAI(
    api_key = OpenAI_KEY,
)

In [148]:
def create_groundtruth(data, client):
    groundtruth_dataset = []
    prompt = """
        You are a course assistant. Given the following text, generate 5 potential questions that could be asked from the content. Return your response as a JSON array of strings, where each string is a question. Do not include any explanations or additional text outside the JSON structure.

        Text: {text_field}

        Response format:
        ["Question 1", "Question 2", "Question 3", "Question 4", "Question 5"]
        """
    for json_document in tqdm(data):
        try:
            text = json_document["text"]
            section = json_document["section"]
            document_id = json_document["document_id"]
            course = json_document["course"]
            prompt_injected = prompt.format(text_field = text)
            completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt_injected,
                }
            ],
                model = 'gpt-4o-mini',
            )
            response = completion.choices[0].message.content
            questions = json.loads(response)
            for question in questions:
                dictionary = dict()
                dictionary["text"] = text
                dictionary["question"] = question
                dictionary["section"] = section
                dictionary["document_id"] = document_id
                dictionary["course"] = course
                groundtruth_dataset.append(dictionary)
        except Exception as e:
            print(e)
            continue        
    return groundtruth_dataset
        

In [149]:



def create_groundtruth(data, client):
    groundtruth_dataset = []
    prompt = """
        You are a course assistant. Given the following text, generate 5 potential questions that could be asked from the content. Return your response as a JSON array of strings, where each string is a question. Do not include any explanations or additional text outside the JSON structure.

        Text: {text_field}

        Response format:
        ["Question 1", "Question 2", "Question 3", "Question 4", "Question 5"]
        """

    @backoff.on_exception(backoff.expo, Exception, max_tries=5)
    def make_api_call(prompt_injected):
        return client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt_injected,
                }
            ],
            model='gpt-4o-mini',
        )

    total_documents = len(data)
    successful_documents = 0

    for i, json_document in enumerate(tqdm(data, desc="Processing documents")):
        try:
            text = json_document["text"]
            section = json_document["section"]
            document_id = json_document["document_id"]
            course = json_document["course"]
            prompt_injected = prompt.format(text_field=text)

            completion = make_api_call(prompt_injected)
            response = completion.choices[0].message.content
            questions = json.loads(response)

            for question in questions:
                dictionary = {
                    "text": text,
                    "question": question,
                    "section": section,
                    "document_id": document_id,
                    "course": course
                }
                groundtruth_dataset.append(dictionary)

            successful_documents += 1

        except json.JSONDecodeError as e:
            print(f"JSON parsing error in document {i}: {e}")
        except Exception as e:
            print(f"Error processing document {i}: {e}")

        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{total_documents} documents. Successful: {successful_documents}")
            time.sleep(75)
    print(f"Finished processing. Total documents: {total_documents}, Successful: {successful_documents}")
    return groundtruth_dataset

In [150]:
groundtruth_dataset = create_groundtruth(json_data, client)

Processing documents:   2%|▏         | 9/435 [00:18<15:20,  2.16s/it]

Processed 10/435 documents. Successful: 10


Processing documents:   4%|▍         | 19/435 [01:59<19:06,  2.75s/it]  

Processed 20/435 documents. Successful: 20


Processing documents:   7%|▋         | 29/435 [03:30<16:51,  2.49s/it]  

...

Processed 430/435 documents. Successful: 421


Processing documents: 100%|█████████▉| 434/435 [1:05:30<00:07,  7.66s/it]

JSON parsing error in document 433: Expecting value: line 1 column 1 (char 0)


Processing documents: 100%|██████████| 435/435 [1:05:32<00:00,  9.04s/it]

Finished processing. Total documents: 435, Successful: 425


In [152]:
with open('groundtruth_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(groundtruth_dataset, f, ensure_ascii=False, indent=4)

## STEP 2 - Create Embeddings using Pretrained Models

In [163]:
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5")

You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [165]:
vector = model.encode("This is a test")
len(vector)

1024

In [181]:
operations = []
for doc in tqdm(json_data):
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

  0%|          | 0/435 [00:00<?, ?it/s]

100%|██████████| 435/435 [2:56:12<00:00, 24.31s/it]  


In [195]:
with open('operations.json', 'w', encoding='utf-8') as f:
    json.dump(operations, f, ensure_ascii=False, indent=4)

### Implementing Elastic Search Retrieval

In [184]:
es = Elasticsearch('http://localhost:9200')

In [185]:
es.info()

ObjectApiResponse({'name': 'baae846d5e1f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'h2iYJMQHQouhleDTcCFGtA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [186]:
index_settings = {
    "settings" : {
        "number_of_shards" : 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 1024, "index" :True, "similarity": "cosine"},
        }
    }
}

In [187]:
index_name = "course-questions"

es.indices.delete(index= index_name, ignore_unavailable=True)
es.indices.create(index= index_name, body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [188]:
for doc in operations:
    try:
        es.index(index= index_name, body= doc)
    except Exception as e:
        print(f"Error is {e}")

In [189]:
search_query = "windows or mac?"
vector_query = model.encode(search_query)

In [190]:
ques = {
    "field" : "text_vector",
    "query_vector": vector_query,
    "k": 5,
    "num_candidates": 1000,
}

In [191]:
res = es.search(index=index_name, knn=ques, source=["text", "section", "question", "course"])


In [192]:
a = res["hits"]["hits"][0]

In [193]:
a.items()

dict_items([('_index', 'course-questions'), ('_id', 'BENg8ZABRJ5_F5RWVrGj'), ('_score', 0.9188564), ('_source', {'question': 'When configuring the profiles.yml file for dbt-postgres with jinja templates with environment variables, I\'m getting "Credentials in profile "PROFILE_NAME", target: \'dev\', invalid: \'5432\'is not of type \'integer\'', 'course': 'data-engineering-zoomcamp', 'section': 'Module 5: pyspark', 'text': 'Update the line:\nWith:'})])

### Milvus Implementation

In [196]:
conn = connections.connect(host="127.0.0.1", port=19530)

database = db.create_database("my_database")


In [197]:
db.using_database("my_database")

In [200]:
from pymilvus import FieldSchema, CollectionSchema, DataType

id_field = FieldSchema(name="document_id", dtype=DataType.INT64, is_primary=True, description="primary id")
text_field = FieldSchema(name='text', dtype= DataType.VARCHAR)
question_field = FieldSchema(name='question', dtype= DataType.VARCHAR)
course_field = FieldSchema(name='course', dtype= DataType.VARCHAR)

embedding_field = FieldSchema(name="text_vector", dtype=DataType.FLOAT_VECTOR, dim=1024, description="vector")


schema = CollectionSchema(fields=[id_field, text_field, question_field, course_field, embedding_field], auto_id=False, enable_dynamic_field=True, description="desc of a collection")


In [201]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)